In [3]:
import os
import re
import pandas as pd

# ─────────────────────────────────────────────────────────────────────────────
# 1) ADJUST THIS PATH to point at your top‐level “NEON_struct-plant” folder:
BASE_DIR = r"/Users/yav13/Work/sdsc/NEON_struct-plant"
# ─────────────────────────────────────────────────────────────────────────────

# 2) We'll walk the entire folder tree under BASE_DIR and collect:
#    - All “vst_mappingandtagging” CSV paths (there may be just one)
#    - All “vst_apparentindividual” CSV paths, keyed by their YYYY-MM tag
#    - All “vst_perplotperyear” CSV paths, keyed by their YYYY-MM tag

mapping_paths = []
ai_paths: dict[str, str] = {}
pp_paths: dict[str, str] = {}

# Helper regex to pull out “YYYY-MM” from filenames like “… .2015-08. …”
month_re = re.compile(r"\.(\d{4}-\d{2})\.")

for root, dirs, files in os.walk(BASE_DIR):
    for f in files:
        if not f.endswith(".csv"):
            continue

        full_path = os.path.join(root, f)

        if "vst_mappingandtagging" in f:
            # All mapping+tagging CSVs (we’ll concat/dedupe later)
            mapping_paths.append(full_path)

        elif "vst_apparentindividual" in f:
            m = month_re.search(f)
            if m:
                ym = m.group(1)  # e.g. "2015-08"
                ai_paths[ym] = full_path

        elif "vst_perplotperyear" in f:
            m = month_re.search(f)
            if m:
                ym = m.group(1)
                pp_paths[ym] = full_path

# 3) Load and combine all “vst_mappingandtagging” CSVs (if there are multiple, keep latest by filename timestamp)
if len(mapping_paths) == 0:
    raise RuntimeError("No vst_mappingandtagging CSV found under BASE_DIR!")

# If there are multiple files, pick the one with highest lexicographic name (assuming latest timestamp)
mapping_paths.sort()
mapping_csv_path = mapping_paths[-1]
df_mt = pd.read_csv(mapping_csv_path, low_memory=False)

# 4) Prepare a list to collect each month’s merged DataFrame
merged_monthly = []

# 5) For each YYYY-MM where both AI and PP exist, merge:
for ym in sorted(set(ai_paths.keys()) & set(pp_paths.keys())):
    ai_file = ai_paths[ym]
    pp_file = pp_paths[ym]

    print(f"Processing month {ym} ->")
    print(f"  • vst_apparentindividual: {ai_file}")
    print(f"  • vst_perplotperyear:     {pp_file}")

    # 5a) Load CSVs
    df_ai = pd.read_csv(ai_file, low_memory=False)
    df_pp = pd.read_csv(pp_file, low_memory=False)

    # 5b) Verify join keys
    #    ai needs at least ["eventID","plotID","individualID"]
    #    pp needs at least ["eventID","plotID"]
    #    mt needs at least ["individualID"]
    for col in ("eventID", "plotID", "individualID"):
        if col not in df_ai.columns:
            raise RuntimeError(f"vst_apparentindividual missing column '{col}' in file {ai_file}")
    for col in ("eventID", "plotID"):
        if col not in df_pp.columns:
            raise RuntimeError(f"vst_perplotperyear missing column '{col}' in file {pp_file}")
    if "individualID" not in df_mt.columns:
        raise RuntimeError(f"vst_mappingandtagging missing column 'individualID' in file {mapping_csv_path}")

    # 5c) Merge AI <- PP on ["eventID","plotID"]
    df_apppp = pd.merge(
        df_ai,
        df_pp,
        on=["eventID", "plotID"],
        how="left",
        suffixes=("_ai", "_pp"),
    )

    # 5d) Merge that result <- MT on ["individualID"]
    df_all = pd.merge(
        df_apppp,
        df_mt,
        on=["individualID"],
        how="left",
        suffixes=("", "_mt"),
    )

    # 5e) Extract exactly the columns requested + “filename”
    #    Columns from vst_apparentindividual:
    #       measurementHeight, stemDiameter, baseCrownHeight
    #    Columns from vst_perplotperyear:
    #       easting, northing, utmZone, decimalLatitude, decimalLongitude
    #    Columns from vst_mappingandtagging:
    #       stemAzimuth, stemDistance
    #    Add a “filename” column equal to the YYYY-MM string
    wanted_cols = [
        "filename",
        "measurementHeight",
        "stemDiameter",
        "baseCrownHeight",
        "easting",
        "northing",
        "utmZone",
        "decimalLatitude",
        "decimalLongitude",
        "stemAzimuth",
        "stemDistance",
    ]

    df_all["filename"] = ym

    # Warn if any expected column is missing (it will appear as all NaN)
    for col in wanted_cols:
        if col not in df_all.columns:
            print(f"  ⚠ Column '{col}' not found after merging {ym}; filling with NaN.")

    df_subset = df_all.reindex(columns=wanted_cols)

    merged_monthly.append(df_subset)

# 6) Concatenate all months into one big DataFrame
if len(merged_monthly) == 0:
    raise RuntimeError("No matching month‐pairs found (vst_apparentindividual & vst_perplotperyear).")

df_merged = pd.concat(merged_monthly, ignore_index=True)

# 7) Write out the final merged CSV
output_path = os.path.join(BASE_DIR, "NEON_all_months_merged.csv")
df_merged.to_csv(output_path, index=False)
print(f"\n✅ Wrote merged file to:\n    {output_path}")


Processing month 2015-08 ->
  • vst_apparentindividual: /Users/yav13/Work/sdsc/NEON_struct-plant/NEON.D17.TEAK.DP1.10098.001.2015-08.basic.20250129T000730Z.RELEASE-2025/NEON.D17.TEAK.DP1.10098.001.vst_apparentindividual.2015-08.basic.20241118T024951Z.csv
  • vst_perplotperyear:     /Users/yav13/Work/sdsc/NEON_struct-plant/NEON.D17.TEAK.DP1.10098.001.2015-08.basic.20250129T000730Z.RELEASE-2025/NEON.D17.TEAK.DP1.10098.001.vst_perplotperyear.2015-08.basic.20241118T024951Z.csv
Processing month 2015-09 ->
  • vst_apparentindividual: /Users/yav13/Work/sdsc/NEON_struct-plant/NEON.D17.TEAK.DP1.10098.001.2015-09.basic.20250129T000730Z.RELEASE-2025/NEON.D17.TEAK.DP1.10098.001.vst_apparentindividual.2015-09.basic.20241118T182352Z.csv
  • vst_perplotperyear:     /Users/yav13/Work/sdsc/NEON_struct-plant/NEON.D17.TEAK.DP1.10098.001.2015-09.basic.20250129T000730Z.RELEASE-2025/NEON.D17.TEAK.DP1.10098.001.vst_perplotperyear.2015-09.basic.20241118T182352Z.csv
Processing month 2015-10 ->
  • vst_apparent

In [8]:
df.columns

Index(['filename', 'measurementHeight', 'stemDiameter', 'baseCrownHeight',
       'easting', 'northing', 'utmZone', 'decimalLatitude', 'decimalLongitude',
       'stemAzimuth', 'stemDistance'],
      dtype='object')

In [13]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('NEON_struct-plant/NEON_all_months_merged.csv')

df = df[['filename', 'measurementHeight', 'stemDiameter',
       'easting', 'northing', 'utmZone', 'decimalLatitude', 'decimalLongitude',
       'stemAzimuth', 'stemDistance']]
print(len(df))
# Remove rows with any missing values
df_cleaned = df.dropna()

# Save the cleaned data to a new CSV file
df_cleaned.to_csv('NEON_TEAK.csv', index=False)

5818


In [14]:
len(df_cleaned)

1907

In [15]:
df_cleaned['measurementHeight_inches'] = df_cleaned['measurementHeight'] * 0.393701
df_cleaned['stemDiameter_inches'] = df_cleaned['stemDiameter'] * 0.393701
df_cleaned['stemDistance_inches'] = df_cleaned['stemDistance'] * 39.3701
df_cleaned.head()

/var/folders/6d/ltzrn7zj535gpgr2_9hj9qn40000gn/T/ipykernel_76409/3797330970.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['measurementHeight_inches'] = df_cleaned['measurementHeight'] * 0.393701
/var/folders/6d/ltzrn7zj535gpgr2_9hj9qn40000gn/T/ipykernel_76409/3797330970.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['stemDiameter_inches'] = df_cleaned['stemDiameter'] * 0.393701
/var/folders/6d/ltzrn7zj535gpgr2_9hj9qn40000gn/T/ipykernel_76409/3797330970.py:3: SettingWithCopyW

,filename,measurementHeight,stemDiameter,easting,northing,utmZone,decimalLatitude,decimalLongitude,stemAzimuth,stemDistance,measurementHeight_inches,stemDiameter_inches,stemDistance_inches
0,2015-08,130.0,10.8,321693.0,4096518.1,11N,36.997917,-119.00382,213.6,1.8,51.18113,4.251971,70.86618
1,2015-08,130.0,54.0,321693.0,4096518.1,11N,36.997917,-119.00382,346.2,16.5,51.18113,21.259854,649.60665
3,2015-08,130.0,31.1,321693.0,4096518.1,11N,36.997917,-119.00382,202.7,3.4,51.18113,12.244101,133.85834
4,2015-08,130.0,28.1,321693.0,4096518.1,11N,36.997917,-119.00382,243.1,7.8,51.18113,11.062998,307.08678
8,2015-08,130.0,12.5,321693.0,4096518.1,11N,36.997917,-119.00382,248.5,4.3,51.18113,4.921263,169.29143


In [16]:
import numpy as np

# Convert azimuth (degrees) to radians
df_cleaned['stemAzimuth_rad'] = np.radians(df_cleaned['stemAzimuth'])

# Compute X and Y
df_cleaned['X'] = df_cleaned['stemDistance'] * np.sin(df_cleaned['stemAzimuth_rad'])
df_cleaned['Y'] = df_cleaned['stemDistance'] * np.cos(df_cleaned['stemAzimuth_rad'])

# Optional: view result
df_cleaned[['decimalLatitude', 'decimalLongitude', 'stemAzimuth', 'stemDistance', 'X', 'Y']].head()

/var/folders/6d/ltzrn7zj535gpgr2_9hj9qn40000gn/T/ipykernel_76409/2204295073.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['stemAzimuth_rad'] = np.radians(df_cleaned['stemAzimuth'])
/var/folders/6d/ltzrn7zj535gpgr2_9hj9qn40000gn/T/ipykernel_76409/2204295073.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['X'] = df_cleaned['stemDistance'] * np.sin(df_cleaned['stemAzimuth_rad'])
/var/folders/6d/ltzrn7zj535gpgr2_9hj9qn40000gn/T/ipykernel_76409/2204295073.py:8: SettingWithCopyWarn

,decimalLatitude,decimalLongitude,stemAzimuth,stemDistance,X,Y
0,36.997917,-119.00382,213.6,1.8,-0.996105,-1.499258
1,36.997917,-119.00382,346.2,16.5,-3.935802,16.023716
3,36.997917,-119.00382,202.7,3.4,-1.312081,-3.136630
4,36.997917,-119.00382,243.1,7.8,-6.956021,-3.528991
8,36.997917,-119.00382,248.5,4.3,-4.000796,-1.575955


In [17]:
df_cleaned.to_csv('NEON_TEAK.csv', index=False)